In [1]:
import qml
import numpy as np
from qml.representations import generate_fchl_acsf
from qml.kernels import get_atomic_local_kernel

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
c = qml.data.compound.Compound('../../test/benzene.xyz')

In [23]:
def distance_sites(repA, repB, siteA, siteB, ZA, ZB):
    sigma = 1 # empirical
    sliceA = np.array([repA])[:,siteA:siteA+1,:]
    sliceB = np.array([repB])[:,siteB:siteB+1,:]
    
    inv_2sigma2 = -1.0 / (2 * sigma**2)
    inv_sigma2 = -1.0 / (sigma**2)
    
    q = get_atomic_local_kernel(sliceA, sliceB, np.array([ZA[siteA:siteA+1]]), np.array([ZB[siteB:siteB+1]]), sigma)[0][0]
    
    # revert transformation
    return np.log(-q/inv_sigma2)/inv_2sigma2

def distance_bn_nb(c, idx_A, idx_B):
    A = c.nuclear_charges.copy()
    A[[idx_A, idx_B]] = (5, 7)
    B = c.nuclear_charges.copy()
    B[[idx_A, idx_B]] = (7, 5)
    repA, gradA = generate_fchl_acsf(A, c.coordinates, gradients=True, pad=len(c.nuclear_charges), elements=range(1, 10))
    repB, gradB = generate_fchl_acsf(B, c.coordinates, gradients=True, pad=len(c.nuclear_charges), elements=range(1, 10))
    
    return distance_sites(repA, repB, idx_A, idx_B, A, B)

distance_bn_nb(c, 0, 11)

1.8346251374335005

In [ ]:
dis